In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
import sys
import os
import base64
from PIL import Image
from io import BytesIO
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import time, base64, json, requests, asyncio
import nest_asyncio
nest_asyncio.apply()
from typing import List, Dict, Union, Any 

# Import our providers
from llm_master import QueryLLM, LLMConfig
config = LLMConfig.from_env()
llm = QueryLLM(config)


/Users/sina/Desktop/Uflo Platform/uflo-AI-server/ai-server/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:llm_master.response_synthesizer:Initialized QueryLLM handler with rate limiters


In [ ]:


async def run_stream():
    messages = [
        {
            "role": "user", 
            "content": "which makes more money software engineer or biomedical engineer as a career?",
        },
    ]

    try:
        response_generator = await llm.query(
            # model_name="o3-mini",
            # reasoning_effort="low",
            model_name="sonar-pro",
            # model_name="claude-3-7-sonnet-latest",
            max_tokens=4096,
            # model_name="accounts/fireworks/models/deepseek-r1",
            messages=messages,
            stream=True,
            fallback_provider="openai",
            fallback_model="gpt-4o",
            moderation=False
        )
        
        async for chunk in response_generator:
            print(chunk, end="", flush=True)
            
    except Exception as e:
        print(f"Error: {str(e)}")

# This line is key in Jupyter
await run_stream()  # Don't use asyncio.run() here

In [ ]:


async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            # model_name="accounts/fireworks/models/deepseek-r1",
            model_name="gemini-2.0-flash",
            # reasoning_effort="low",
            messages=messages,
            stream=False,  # Set to False for non-streaming
            # temperature=0.5,
            fallback_provider="openai",
            fallback_model="gpt-4o",
            moderation=False
        )
        
        # Print the full response content
        print(response.content)
        
        # You can also access other metadata
        print("\n--- Response Metadata ---")
        print(f"Model: {response.model_name}")
        print(f"Input tokens: {response.usage.input_tokens}")
        print(f"Output tokens: {response.usage.output_tokens}")
        print(f"Cost: ${response.cost:.6f}")
        print(f"Latency: {response.latency:.2f} seconds")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [3]:

async def test_perplexity():
    # Sample messages
    messages = [
        {
            "role": "system",
            "content": "You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user. Answer as concisely as possible."
        },
        {   
            "role": "user",
            "content": "How many stars are in the universe?"
        },
    ]
    
    # Test non-streaming query
    print("Testing non-streaming Perplexity API with citations")
    response = await llm.query(
        model_name="sonar",
        messages=messages,
        stream=False
    )
    
    print(f"Response content: {response.content}")
    # print(f"Citations: {response.citations}")
    
    # Test streaming query
    print("\nTesting streaming Perplexity API with citations")
    stream_generator = await llm.query(
        model_name="sonar",
        messages=messages,
        stream=True
    )
    
    full_response = ""
    citations_found = False
    
    async for chunk in stream_generator:
        if chunk.startswith("\n<citations>"):
            print(f"Found citations in stream: {chunk}")
            citations_found = True
        else:
            full_response += chunk
            print(f"Received chunk: {chunk}")
    
    print(f"\nFull response: {full_response[:100]}...")
    
    # After streaming is complete, check if provider has citations
    provider = llm._get_provider("sonar")
    if hasattr(provider, 'last_citations') and provider.last_citations:
        print(f"Citations from provider.last_citations: {provider.last_citations}")
        citations_found = True
    
    if not citations_found:
        print("No citations found in streaming response")

await test_perplexity()


Testing non-streaming Perplexity API with citations


INFO:llm_master.base_provider:Initialized perplexity provider with base URL: https://api.perplexity.ai
INFO:llm_master.base_provider:Successfully initialized perplexity provider
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"


Response content: Estimating the total number of stars in the universe is a complex task. Recent estimates suggest there are approximately **200 sextillion (200 billion trillion)** stars, although some sources suggest a slightly lower figure around **70 billion trillion** stars in the observable universe[1][3]. These estimates depend on the assumption of an average galaxy having about 100 billion stars and the estimated number of galaxies, which is now thought to be around 2 trillion[1][3]. However, these numbers can vary based on the methods used and the assumptions made about average galaxy size and star count.
<citation_list><citation_source>https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/</citation_source><citation_source>https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe</citation_source><citation_source>https://skyandtelescope.org/astronomy-resources/how-many-stars-are-there/</citation_source>

INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"


Received chunk: Est
Received chunk: imating the
Received chunk:  exact number
Received chunk:  of stars
Received chunk:  in the
Received chunk:  universe is
Received chunk:  a complex
Received chunk:  task,
Received chunk:  but astronomers
Received chunk:  have provided
Received chunk:  various estimates
Received chunk: . One
Received chunk:  estimate suggests
Received chunk:  there are
Received chunk:  approximately **
Received chunk: 200 sext
Received chunk: illion (
Received chunk: 200 billion
Received chunk:  trillion)**
Received chunk:  stars in
Received chunk:  the universe
Received chunk: , based
Received chunk:  on multiplying
Received chunk:  the estimated
Received chunk:  number of
Received chunk:  stars in
Received chunk:  a typical
Received chunk:  galaxy (
Received chunk: 100 billion
Received chunk: ) by
Received chunk:  the number
Received chunk:  of galaxies
Received chunk:  (2
Received chunk:  trillion)[1].
Received chunk:  However,
Received chunk:  another estimate
Rec

In [ ]:
INFO:llm_master.base_provider:Initialized perplexity provider with base URL: https://api.perplexity.ai
INFO:llm_master.base_provider:Successfully initialized perplexity provider
Testing non-streaming Perplexity API with citations
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
Response content: Estimating the total number of stars in the universe is a complex task, but astronomers provide several estimates based on observable data. Some sources suggest there are about **200 billion trillion** (200 sextillion) stars across the universe, while others place the number at around **70 billion trillion** (7 x 10^22) within the observable universe alone[1][3]. However, these estimates vary due to factors like the vastness of space, variations in galaxy sizes, and our limited ability to observe distant stars. Another estimate suggests about **2.14 sextillion** stars, adjusting for the fact that not all galaxies are as large as the Milky Way and accounting for stellar lifecycles[5].
<citation_list><citation_source>https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/</citation_source><citation_source>https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe</citation_source><citation_source>https://skyandtelescope.org/astronomy-resources/how-many-stars-are-there/</citation_source><citation_source>https://www.youtube.com/watch?v=xtgJc1y6kq0</citation_source><citation_source>https://bigthink.com/starts-with-a-bang/overestimated-stars-in-universe/</citation_source></citation_list>
Citations: ['https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/', 'https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe', 'https://skyandtelescope.org/astronomy-resources/how-many-stars-are-there/', 'https://www.youtube.com/watch?v=xtgJc1y6kq0', 'https://bigthink.com/starts-with-a-bang/overestimated-stars-in-universe/']

Testing streaming Perplexity API with citations
INFO:httpx:HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
Received chunk: 
<citation_list><citation_source>https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/</citation_source><citation_source>https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe</citation_source><citation_source>https://skyandtelescope.org/astronomy-resources/how-many-stars-are-there/</citation_source><citation_source>https://www.youtube.com/watch?v=xtgJc1y6kq0</citation_source><citation_source>https://bigthink.com/starts-with-a-bang/overestimated-stars-in-universe/</citation_source></citation_list>

Full response: 
<citation_list><citation_source>https://www.astronomy.com/science/astro-for-kids-how-many-stars-are...
Citations from provider.last_citations: ['https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/', 'https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe', 'https://skyandtelescope.org/astronomy-resources/how-many-stars-are-there/', 'https://www.youtube.com/watch?v=xtgJc1y6kq0', 'https://bigthink.com/starts-with-a-bang/overestimated-stars-in-universe/']

In [4]:

# Using messages format
response1 = await llm.query(
    model_name="imagen-3.0-generate-002",
    messages=[{"role": "user", "content": """Create a detailed 6-panel storyboard arranged in 3 rows of 2 columns. Each panel should be clearly separated. Art style should be bright, vibrant, and slightly stylized with a summery color palette dominated by blues, yellows, and warm tones. The style should be suitable for family audiences, capturing both the fun and serenity of a beach day.

Break down of key scenes:

## Panel 1:
A family of five arrives at Sunny Shores Beach early morning, with the parents unloading beach gear from their car while the three children (older boy Daniel, middle child narrator, and little brother Leo) gaze excitedly at the expansive sandy beach with palm trees swaying against a bright blue sky dotted with fluffy white clouds.

## Panel 2:
The family's beach setup is complete with bright blue towels and red blankets under a large umbrella providing shade, while nearby Leo digs holes near the shoreline with his small shovel, the middle child builds a large sandcastle, and Daniel explores near a tall lifeguard tower in the background.

## Panel 3:
Daniel rides a boogie board on a blue-green wave, his expression joyful as he navigates the water, while in the foreground the middle child and Leo splash in the shallower water, and their mother can be seen collecting seashells along the shoreline.

## Panel 4:
Father sits quietly fishing from a rocky section of the shore with several other fishermen visible along the coastline, while in the background hikers can be seen on paths near the distant mountains, and the family's umbrella and setup is visible with mom now relaxing in the shade.

## Panel 5:
The family walks through a beachside area with shops and restaurants, all holding refreshing mango drinks with juice dripping down, while Leo points excitedly at dogs playing in a small dog park area adjacent to a playground.

## Panel 6:
As the sun begins to set, turning the clouds into beautiful orange and pink hues, the family walks along the shoreline together, leaving footprints in the wet sand, with their silhouettes cast against the colorful sky and ocean reflection.

## Key details to maintain throughout:
- Character design consistency: Family of five (mom, dad, teenage boy Daniel, middle child narrator, young boy Leo)
- Color palette: Vibrant blues of ocean (dark blue to blue-green gradient), warm beige sand, bright sky, sunset colors in final panel
- Environmental continuity: Palm trees, mountains in background, combination of sandy and rocky areas
- Important recurring elements: Beach umbrella, towels/blankets as home base
- Style-specific requirements: Clean lines with vibrant colors, slight stylization without being cartoonish
- Mood and atmosphere: Joyful, relaxed family day transitioning from bright daylight to golden sunset hour"""}],
)
image = Image.open(BytesIO(base64.b64decode(response1.content)))
image.show()

INFO:llm_master.base_provider:Generating image with Imagen provider. Base URL: https://generativelanguage.googleapis.com/v1beta/openai/


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/images/generations "HTTP/1.1 200 OK"


In [5]:
print(response1.content)

...


In [ ]:
# Define the prompt
prompt = "A cat on its back legs running like a human is holding a big silver fish with its arms. The cat is running away from the shop owner and has a panicked look on his face. The scene is situated in a crowded market."

try:
    # Generate the image
    response = await llm.query(
        model_name="recraftv3",
        messages=[{"role": "user", "content": prompt}],
        style="digital_illustration"
    )
    
    # Print the result (which is the image URL)
    print(f"Image generation successful! URL: {response.content}")
    
except Exception as e:
    print(f"Error generating image: {str(e)}")

In [6]:
# Define the prompt
prompt = """Create a detailed 6-panel storyboard arranged in 3 rows of 2 columns. Each panel should be clearly separated. Art style should be bright, vibrant, and slightly stylized with a summery color palette dominated by blues, yellows, and warm tones. The style should be suitable for family audiences, capturing both the fun and serenity of a beach day.

Break down of key scenes:

## Panel 1:
A family of five arrives at Sunny Shores Beach early morning, with the parents unloading beach gear from their car while the three children (older boy Daniel, middle child narrator, and little brother Leo) gaze excitedly at the expansive sandy beach with palm trees swaying against a bright blue sky dotted with fluffy white clouds.

## Panel 2:
The family's beach setup is complete with bright blue towels and red blankets under a large umbrella providing shade, while nearby Leo digs holes near the shoreline with his small shovel, the middle child builds a large sandcastle, and Daniel explores near a tall lifeguard tower in the background.

## Panel 3:
Daniel rides a boogie board on a blue-green wave, his expression joyful as he navigates the water, while in the foreground the middle child and Leo splash in the shallower water, and their mother can be seen collecting seashells along the shoreline.

## Panel 4:
Father sits quietly fishing from a rocky section of the shore with several other fishermen visible along the coastline, while in the background hikers can be seen on paths near the distant mountains, and the family's umbrella and setup is visible with mom now relaxing in the shade.

## Panel 5:
The family walks through a beachside area with shops and restaurants, all holding refreshing mango drinks with juice dripping down, while Leo points excitedly at dogs playing in a small dog park area adjacent to a playground.

## Panel 6:
As the sun begins to set, turning the clouds into beautiful orange and pink hues, the family walks along the shoreline together, leaving footprints in the wet sand, with their silhouettes cast against the colorful sky and ocean reflection.

## Key details to maintain throughout:
- Character design consistency: Family of five (mom, dad, teenage boy Daniel, middle child narrator, young boy Leo)
- Color palette: Vibrant blues of ocean (dark blue to blue-green gradient), warm beige sand, bright sky, sunset colors in final panel
- Environmental continuity: Palm trees, mountains in background, combination of sandy and rocky areas
- Important recurring elements: Beach umbrella, towels/blankets as home base
- Style-specific requirements: Clean lines with vibrant colors, slight stylization without being cartoonish
- Mood and atmosphere: Joyful, relaxed family day transitioning from bright daylight to golden sunset hour."""

try:
    # Generate the image
    response = await llm.query(
        model_name="flux-dev",
        messages=[{"role": "user", "content": prompt}],
        width=1024,  # Optional: specify image width
        height=768,   # Optional: specify image height
        prompt_upsampling=True, #If active, automatically modifies the prompt for more creative generation.
        guidance=3, #High guidance scales improve prompt adherence at the cost of reduced realism.
        fallback_provider="gemini",
        fallback_model="imagen-3.0-generate-002",
    )
    
    # Print the result (which is the image URL)
    if response.content.startswith("http"):
        # It's a URL - print it
        print(f"Image generation successful! URL: {response.content}")
    else:
        # It's base64 - decode and show
        import base64
        from io import BytesIO
        from PIL import Image
        
        image = Image.open(BytesIO(base64.b64decode(response.content)))
        image.show()
        print("Displayed base64 encoded image")
    
except Exception as e:
    print(f"Error generating image: {str(e)}")

INFO:llm_master.bfl_provider:BFL image generation initiated with request ID: 4833a2fd-7fa8-4196-a95c-408e6ef29aa6
INFO:llm_master.response_synthesizer:Using fallback provider gemini with model imagen-3.0-generate-002
INFO:llm_master.base_provider:Generating image with Imagen provider. Base URL: https://generativelanguage.googleapis.com/v1beta/openai/


Displayed base64 encoded image


In [ ]:

async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            model_name="gpt-4o-mini-audio-preview",
            messages=messages,
            stream=False,
            modality=["text", "audio"],
            audio={"voice": "ash", "format": "wav"}
        )
        
         # Print the text response
        print("Text response:")
        print(response.content)
        
        # Save the audio to a file if available
        if response.audio_data:
            wav_bytes = base64.b64decode(response.audio_data)
            output_file = "dog_response.wav"
            with open(output_file, "wb") as f:
                f.write(wav_bytes)
            print(f"\nAudio saved to '{output_file}'")
        else:
            print("\nNo audio data received in the response")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [ ]:

async def run_query():
    messages = [
        {
            "role": "user", 
            "content": f"Narrate the text inside <text_to_read> tags in an authentic ${language.name} like a native from ${language.capital}. Only read the text inside the tags and nothing else. <text_to_read> ${formattedText} </text_to_read> "
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            model_name="gpt-4o-mini-audio-preview",
            messages=messages,
            stream=False,
            modality=["text", "audio"],
            audio={"voice": "ash", "format": "wav"}
        )
        
         # Print the text response
        print("Text response:")
        print(response.content)
        
        # Save the audio to a file if available
        if response.audio_data:
            wav_bytes = base64.b64decode(response.audio_data)
            output_file = "dog_response.wav"
            with open(output_file, "wb") as f:
                f.write(wav_bytes)
            print(f"\nAudio saved to '{output_file}'")
        else:
            print("\nNo audio data received in the response")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()